In [8]:
import collections
from collections import defaultdict
from sklearn import svm
from sklearn import cross_validation
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import pickle
import json

In [9]:
# specify dir
Dir = 'D:/Depression_project/data/twitter'

# Load data

In [38]:
texts = []
ids = []
#with open(Dir+'/'+'tweets_cleaned.txt',"r", errors = 'ignore') as f:
with open(Dir+'/'+'tweetsEng_cleaned_2.txt',"r", errors = 'ignore') as f:
    data = f.readlines()
texts = [x.strip().split('#^~^#')[1] for x in data] 
ids = [x.strip().split('#^~^#')[0] for x in data] 
len(texts)

171514

In [4]:
print (texts[:2])
print (ids[:2])

['partli cloudi night temp:3.4c hum:87 wind ene mph maxgust:10 mph rain mm baro:1010.6 hpa amp rise slowli weather', 'rt date would enjoy']
['980280726804467713', '980280726796013568']


# Clean data

In [5]:
def get_words(docs):
    words = []
    for doc in docs:
        for word in doc.split():
            words.append(word)
    print ("The number of unique words: "+ str(len(set((words)))))
    return words

# define a funciton to find words occurring less than n times and delete that word
def findWordLessThan(counter, n):
    uncommon = []
    for k, v, in counter.items():
        if v < n:
            uncommon.append(k)            
    print (len(uncommon))
    return uncommon

def findWordMoreThan(counter, n):
    common = []
    for k, v, in counter.items():
        if v > n:
            common.append(k)
    print (len(common))
    return common

In [27]:
words = get_words(texts)
count = collections.Counter(words)

The number of unique words: 127588


In [28]:
uncommon = findWordLessThan(count, 6)
common = findWordMoreThan(count, 100)

112894
1895


In [30]:
# check
for k, v in count.items():
    if k in common:
        print (k, v)

lil 439
north 177
station 261
@joyannreid 103
apolog 255
backup 152
mean 1465
emot 443
grant 102
clean 218
sweet 336
massiv 116
teenag 116
sister 334
@anthonyfjoshua 122
hype 108
invit 182
white 1000
b 524
@keetbe 122
guest 197
fortnit 307
lvl 139
crime 170
hi 5013
retweet 1074
rate 256
data 105
wit 218
experi 415
same 1256
exclus 184
discov 127
abt 103
imma 119
sub 149
medal 133
iphon 104
boss 101
live 2019
| 1085
student 546
non 469
africa 123
club 479
injuri 127
ikon 104
airport 111
tl 143
beauti 1275
global 147
timelin 119
championship 111
seem 460
allow 309
doubt 130
blood 253
code 167
never 2666
° 220
@nicoletatum26 143
kerr 120
gorgeou 143
have 18910
poll 159
british 158
vinc 217
# 17589
guid 155
quit 242
bore 248
depress 261
also 1322
bill 235
snail 113
yodel 198
hillari 130
yet 500
probabl 495
adult 217
sport 192
album 650
tweet 1782
busi 772
month 1158
sing 656
1st 499
surround 427
q 112
seek 173
rain 199
ish 103
exchang 113
trash 171
leader 385
camera 206
insult 209
welcom 5

In [31]:
corpus = list(set(count.keys()) - set(uncommon))
print (len(corpus))

14694


In [32]:
messy = ['#', '’', '@', 'rt', '_',  'http', '%', 'x', '…', '|', '/', '▔', 'amp', '┏', '★', '═', '&gt', '╚', '━', '║', '『', \
        '╮','°', '―']
messy2 = ['a','b','c','d','e','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
messy_common = []
for w in corpus:
    for i in messy:
        if w.find(i) != -1 and w not in messy2:
            messy_common.append(w)
print (len(messy_common))

4082


In [33]:
corpus = list(set(count.keys()) - set(uncommon) - set(messy_common))
print (len(corpus))

11254


In [37]:
for k, v in collections.Counter(corpus).items():
    if k in corpus[3000:4000]:
        print (k)

limitless
bankrupt
feb
length
tit
leicest
creepi
produ
shite
just
u.s.
keith
supernatur
yeontan
massacr
agreement
wesley
dbfz
entrata
berserk
easterbunni
adjust
dd
imposit
gerri
fargo
dedic
unifi
hooker
ic
someth
clinch
true
cinemat
niggaz
russ
qualif
grass
esskeetit
elf
farewel
jerusalem
keller
fluid
merg
poc
plight
rig
og
stain
peek
perf
bat
negara
white-
postag
effigi
fin
flyer
rosann
guinea
midst
mermaid
puddl
karaok
kiyoko
vijay
chavez
lmfaoooooo
album
phenom
memento
accus
devoid
volleybal
chap
jumla
hind
nurseri
provinc
turd
aspir
redmayn
tweet
busi
owner
sustain
almig
bossman
glen
prior
consist
tuff
mail
cuddl
delta
loo
evolv
lit
sturgeon
notion
anita
charm
cher
pi
enslav
cyru
hoggwash
month
unabl
moomoo
lololol
contagi
sing
blackmail
alyson
custom
finn
it~
shoulda
algorithm
duct
njsg
1st
essenc
salad
stil
bottom
lo
inspir
iq
hellcat
sunris
babi
instal
profoundest
prob
gaurav
hat
lama
quot
tantrum
kokobop
kylo
syrian
tommi
wbo
emir
romanc
brigad
porno
mnet
lifelong
bigmouth
vict

# Bag of words

In [11]:
def get_sparse_vec(docs, word_space):
    sparse_vecs = []
    for doc in docs:
        # create empty vector
        sparse_vec = np.zeros(len(word_space))
        for word in doc.split():
            if word in word_space:
                sparse_vec[word_space[word]] = 1
        sparse_vecs.append(sparse_vec) 
    print ("The number of samples is " +str(len(sparse_vecs)))
    if len(sparse_vecs[0]) != len(word_space):
        print ("error")
    return sparse_vecs

In [12]:
# load word_space 
with open("D:/Depression_project/word_space.json", mode='r') as file:
    word_space = json.load(file)   # json.load reads whole json file, json.loads reads string only
print (len(word_space))

16744


In [39]:
sparse_vecs = get_sparse_vec(texts, word_space)
print (len(sparse_vecs[0]))
print (sparse_vecs[0])

MemoryError: 

In [14]:
# load saved SVM classifier
filename = 'D:/Depression_project/svmModel.sav'
svmModel = pickle.load(open(filename, 'rb'))

In [17]:
def findCase(svmClf, inputX, cutoff, outfile):
    f = open(outfile, "w")
    #label = []
    count = 0
    predP = svmClf.predict_proba(inputX)
    for p in predP:
        if p[1] >= cutoff:
            #label.append((count, 1))
            f.write(str(count)+' 1'+'\n')
        else:
            #label.append((count, 0))
            f.write(str(count)+' 0'+'\n')
        count += 1 
    f.close()
    #return label

In [ ]:
def findProb(svmClf, inputX, outfile):  
    count = 0
    predP = svmClf.predict_proba(inputX)
    f = open(outfile, "w")
    for p in predP:
        f.write(str(count)+str(round(p[1]), 2)+'\n')
        count += 1 
    f.close()

In [18]:
newX = sparse_vecs[:1000]
label = findCase(svmModel, newX, 0.8, Dir+'/'+'predictedProb.txt')

In [ ]:
import datetime
start = datetime.datetime.now().replace(microsecond=0)
newX = sparse_vecs[:10000]
end = datetime.datetime.now().replace(microsecond=0)
print (end-start)

In [20]:
#predY = svmModel.predict(newX)
predP = svmModel.predict_proba(newX)

In [24]:
Y = []
for p in predP:
    if p[1] >= 0.85:
        Y.append(1)
    else:
        Y.append(0)
print ("The number of possible depression expressions is "+ str(sum(Y)))

The number of possible depression expressions is 5


In [29]:
count = -1
for i in Y:
    count += 1
    if i == 1:
        print (count)
        print (texts[count])

1794
rt offici will be togeth long time order reach what start put arsen be deserv spot
2758
rt @pvpi_daij get point where do not wanna nobodi nomor just leav alon would rather distanc
4973
rt @exokiss88 changmin say will be great exo shine tvqx super junior member can have fun togeth do fun program tog
6027
rt real make # nomakeupchalleng
7962
eat be not stupid be leftist ano ba di fam


In [34]:
len(predP)

1000

In [33]:
f = open(Dir+'/'+'predictedProb.txt', "w")
for row in predP:
    f.write(str(row[1])+'\n')
f.close()

In [5]:
for i in range(0, 104, 20):
    print ((i, i+20))

(0, 20)
(20, 40)
(40, 60)
(60, 80)
(80, 100)
(100, 120)


In [10]:
labels = []
ids = []
with open(Dir+'/'+'predictedProb_0.txt',"r", errors = 'ignore') as f:
    data = f.readlines()
labels = [x.strip().split()[1] for x in data] 
ids = [x.strip().split()[0] for x in data] 
len(labels)

2000